In [4]:
import os

import pandas as pd

In [5]:
seed = 42
data_path = '/nobackup/users/hmbaghda/metastatic_potential/'

In [56]:
expr_protein = pd.read_csv(os.path.join(data_path, 'processed',  'expr_protein.csv'), index_col = 0)
mp_protein = pd.read_csv(os.path.join(data_path, 'processed', 'metastatic_potential_protein.csv'), index_col = 0)

expr_rna = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'), index_col = 0)
mp_rna = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'))

md = pd.read_csv(os.path.join(data_path, 'raw', 'Model.csv'), index_col = 0)


In [59]:
expr_protein.index = pd.Series(expr_protein.index).apply(lambda x: x.split('_')[0])

sample_mapper = md[md.StrippedCellLineName.isin(expr_protein.index)]
sample_mapper = dict(zip(sample_mapper.StrippedCellLineName, sample_mapper.index))
if len(sample_mapper) != expr_protein.shape[0]:
    raise ValueError('Expect all samples to be mapped')
expr_protein.index = expr_protein.index.map(sample_mapper)
mp_protein.index = expr_protein.index

In [65]:
common_samples = sorted(set(expr_protein.index).intersection(expr_rna.index))

print('Of the {} and {} samples in protein and RNA datasets, respectively, {} are shared'.format(expr_protein.shape[0], 
                                                                                                expr_rna.shape[0], 
                                                                                                len(common_samples)))

Of the 251 and 481 samples in protein and RNA datasets, respectively, 250 are shared


In [66]:
expr_protein = expr_protein.loc[common_samples, :]
expr_rna = expr_rna.loc[common_samples, :]
mp = mp_protein.loc[common_samples, :]

expr = pd.concat([expr_protein, expr_rna], axis = 1, ignore_index = False)
mp.to_csv(os.path.join(data_path, 'processed', 'metastatic_potential_joint.csv'))
expr.to_csv(os.path.join(data_path, 'processed', 'expr_joint.csv'))